In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob

import mne
from mne.datasets import spm_face
from mne.preprocessing import ICA, create_eog_epochs
from mne import io, combine_evoked
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP, UnsupervisedSpatialFilter)

mne.set_log_level('warning')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA


In [3]:
def model(epochs, label):

    X = epochs.get_data()
    #print(X.shape)
    # X_scaled = scaler.fit_transform(X.reshape(len(labels), -1))

    y = label   
    
    scaler = Scaler(scalings = 'mean')
    svm  = SVC(C= 1,gamma = 0.001)
    csp = CSP(n_components=3, norm_trace=False)
    pca = UnsupervisedSpatialFilter(PCA(n_components = 5, svd_solver = 'full'), average=False)
    clf = Pipeline(steps=[
        ('scaler', scaler),
        ('csp', csp),
        #('pca', pca),
        ('vectorizer', Vectorizer()),
        ('svm', svm)
    ])
    
    scores = cross_val_multiscore(clf, X, y, cv=5, n_jobs=1).mean()
    print(scores)
    
    return X, y, clf

In [4]:
def score(X, y, clf, n):
            
    time_decod = SlidingEstimator(clf, n_jobs=1, scoring='roc_auc', verbose=False)
    scores = cross_val_multiscore(time_decod, X, y, cv=5, n_jobs=1)

    # Mean scores across cross-validation splits
    scores = np.mean(scores, axis=0)

    # Plot
    fig, ax = plt.subplots()
    ax.plot(epochs.times, scores, label='score')
    ax.axhline(.5, color='k', linestyle='--', label='chance')
    ax.set_xlabel('Times')
    ax.set_ylabel('AUC')  # Area Under the Curve
    ax.legend()
    ax.axvline(.0, color='k', linestyle='-')
    ax.set_title('Sensor space decoding participant {}'.format(n))
    plt.show()

In [ ]:
allepochs = []
alllabels = []

for i in all_participants:
    participant = i
    
    epochs_subtracted = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'ar_subtracted_epo.fif') 
    epochs_subtracted.info['description'] = 'biosemi/64' #necessary for wSMI 
    epochs_subtracted =  epochs_subtracted.pick_types(eeg = True) #EOGs break everything\
    
    allepochs.append(mne.epochs.concatenate_epochs(epochs_subject, add_offset = True))
    alllabels.append(np.array(y))
    
    
    


In [ ]:
for i in all_participants:
    participant = i
    
  

    folder = path + participant +'/'

    
    df_markers = pd.DataFrame()
    df_markers['participant'] = participant
    
    #############################
    #### With ERP SUBTRACTED ####
    #############################
    epochs_subtracted = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'ar_subtracted_epo.fif') 
    epochs_subtracted.info['description'] = 'biosemi/64' #necessary for wSMI 
    epochs_subtracted =  epochs_subtracted.pick_types(eeg = True) #EOGs break everything\